In [ ]:
#!/usr/bin/env python3
import sqlite3
import os
import argparse
import sys
from pathlib import Path

def copy_table_subset(source_db, destination_db, table_name, limit=200000):
    """
    Copy the first 'limit' rows from a table in the source database to a new database.

    Args:
        source_db (str): Path to the source SQLite database file
        destination_db (str): Path to create the destination SQLite database file
        table_name (str): Name of the table to copy
        limit (int): Maximum number of rows to copy (default: 200000)
    """
    # Verify source database exists
    if not os.path.exists(source_db):
        sys.exit(f"Error: Source database '{source_db}' does not exist")

    # Create destination directory if it doesn't exist
    dest_dir = os.path.dirname(destination_db)
    if dest_dir and not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # If destination already exists, confirm overwrite
    if os.path.exists(destination_db):
        response = input(f"Destination database '{destination_db}' already exists. Overwrite? (y/n): ")
        if response.lower() != 'y':
            sys.exit("Operation cancelled")
        os.remove(destination_db)

    try:
        # Connect to source database with recovery mode
        source_conn = sqlite3.connect(f"file:{source_db}?immutable=1", uri=True)
        source_cursor = source_conn.cursor()

        # Verify the table exists in source database
        source_cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
        if not source_cursor.fetchone():
            sys.exit(f"Error: Table '{table_name}' does not exist in source database")

        # Get table schema
        source_cursor.execute(f"PRAGMA table_info({table_name})")
        columns_info = source_cursor.fetchall()
        column_names = [info[1] for info in columns_info]
        columns_str = ', '.join([f'"{name}"' for name in column_names])
        placeholders = ', '.join(['?' for _ in column_names])

        # Create destination database
        dest_conn = sqlite3.connect(destination_db)
        dest_cursor = dest_conn.cursor()

        # Setup custom text factory to handle encoding errors
        def text_factory(x):
            try:
                return str(x, 'utf-8')
            except UnicodeDecodeError:
                return str(x, 'utf-8', errors='replace')

        source_conn.text_factory = text_factory
        dest_conn.text_factory = text_factory

        # First, try to get the CREATE TABLE statement from source
        try:
            source_cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name=?", (table_name,))
            create_table_sql = source_cursor.fetchone()[0]
            dest_cursor.execute(create_table_sql)
        except Exception as e:
            print(f"Warning: Could not copy original table schema: {e}")
            print("Attempting to recreate schema based on PRAGMA table_info...")

            # Fallback: manually create table based on column info
            create_stmt = [f"CREATE TABLE {table_name} ("]
            cols = []
            for col in columns_info:
                col_id, col_name, col_type, notnull, default_val, is_pk = col
                col_def = f'"{col_name}" {col_type}'
                if notnull:
                    col_def += " NOT NULL"
                if default_val is not None:
                    col_def += f" DEFAULT {default_val}"
                if is_pk:
                    col_def += " PRIMARY KEY"
                cols.append(col_def)
            create_stmt.append(", ".join(cols))
            create_stmt.append(")")
            create_table_sql = " ".join(create_stmt)

            print(f"Creating table with: {create_table_sql}")
            dest_cursor.execute(create_table_sql)

        # Begin transaction for better performance
        dest_conn.execute("BEGIN TRANSACTION")

        # Query for first 'limit' rows
        source_cursor.execute(f"SELECT {columns_str} FROM {table_name} LIMIT ?", (limit,))

        # Fetch and insert rows in batches for memory efficiency
        batch_size = 10000
        count = 0
        error_count = 0
        max_errors = 100  # Maximum number of errors to tolerate

        while True:
            try:
                rows = source_cursor.fetchmany(batch_size)
                if not rows:
                    break

                # Process each row individually to handle errors
                for row in rows:
                    try:
                        dest_cursor.execute(
                            f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders})",
                            row
                        )
                        count += 1
                    except sqlite3.Error as e:
                        error_count += 1
                        if error_count <= 5:  # Only show first few errors
                            print(f"\nError inserting row: {e}")
                        if error_count >= max_errors:
                            print(f"\nToo many errors ({error_count}). Stopping.")
                            break

                    if count % 100 == 0:
                        print(f"\rCopied {count} rows... (Errors: {error_count})", end="", flush=True)

                if count >= limit or error_count >= max_errors:
                    break

            except sqlite3.Error as e:
                print(f"\nError in batch: {e}")
                error_count += 1
                if error_count >= max_errors:
                    print(f"\nToo many errors ({error_count}). Stopping.")
                    break

        # Commit the transaction
        dest_conn.commit()

        # Try to copy indexes (best effort)
        try:
            source_cursor.execute(
                "SELECT sql FROM sqlite_master WHERE sql IS NOT NULL AND type='index' "
                "AND tbl_name=? AND name NOT LIKE 'sqlite_%'",
                (table_name,)
            )
            for row in source_cursor.fetchall():
                if row[0]:  # Skip if sql is None
                    try:
                        dest_cursor.execute(row[0])
                    except sqlite3.Error as e:
                        print(f"Warning: Could not create index: {e}")
        except sqlite3.Error as e:
            print(f"Warning: Could not copy indexes: {e}")

        # Optimize the database
        try:
            dest_cursor.execute("PRAGMA optimize")
        except sqlite3.Error:
            pass  # Ignore if not supported

        if error_count > 0:
            print(f"\nPartially copied {count} rows with {error_count} errors from '{table_name}' to '{destination_db}'")
        else:
            print(f"\nSuccessfully copied {count} rows from '{table_name}' to '{destination_db}'")

    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
        # If there was an error and we created a new destination file, clean it up
        if not os.path.exists(source_db) and os.path.exists(destination_db):
            os.remove(destination_db)
        return False
    except Exception as e:
        print(f"Error: {e}")
        return False

    finally:
        # Close all connections
        if 'source_conn' in locals():
            source_conn.close()
        if 'dest_conn' in locals():
            dest_conn.close()

In [ ]:
source_db = "/mnt/z/Uni/Master Thesis/eyeBOLD/eyeBOLD_db.db"
destination_db = "/mnt/z/Uni/Master Thesis/eyeBOLD/eyeBOLD_mini2.db"
table_name = "specimen"
limit = 200000

copy_table_subset(source_db, destination_db, table_name, limit)

In [ ]:
db = sqlite3.connect(destination_db)
cursor = db.cursor()

try:
    # Check if table exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='primer_matches'")
    table_exists = cursor.fetchone() is not None

    # Create the table if it doesn't exist
    if not table_exists:
        cursor.execute("""
            CREATE TABLE primer_matches (
                match_id INTEGER PRIMARY KEY AUTOINCREMENT,
                specimen_id INTEGER NOT NULL,
                primer_sequence TEXT NOT NULL,
                primer_start_index INTEGER,
                primer_end_index INTEGER,
                match_score FLOAT,
                FOREIGN KEY (specimen_id) REFERENCES specimen(specimenid),
                UNIQUE (specimen_id, primer_sequence)
            )
        """)
        # Commit immediately after CREATE TABLE
        db.commit()
        print("Table created successfully")
finally:
    # Always close the connection
    cursor.close()
    db.close()
    print("Connection closed")